# Implementación de un Modelo de Clasificación de Vinos como Servicio Web con FastAPI

Este notebook documenta el proceso de implementación de un modelo de clasificación de vinos como servicio web utilizando FastAPI. Incluiremos el código completo, ejemplos de uso y pruebas del servicio.

## 1. Instalación de Dependencias

Primero, instalamos las bibliotecas necesarias:

In [ ]:
!pip install fastapi uvicorn numpy pandas scikit-learn pydantic python-multipart loguru

## 2. Implementación del Modelo de Clasificación

Implementamos un modelo de clasificación para predecir la calidad del vino usando scikit-learn:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import joblib

def train_model():
    # Cargamos el dataset de vinos
    data = pd.read_csv('wine.data', header=None)
    
    # Separamos features y target
    X = data.iloc[:, 1:].values
    y = data.iloc[:, 0].values
    
    # Dividimos los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Escalamos las características
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Creamos y entrenamos el modelo
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42
    )
    model.fit(X_train_scaled, y_train)
    
    # Evaluamos el modelo
    score = model.score(X_test_scaled, y_test)
    print(f'Precisión del modelo: {score:.4f}')
    
    # Guardamos el modelo y el scaler
    joblib.dump(model, 'classifier_model.joblib')
    joblib.dump(scaler, 'scaler.joblib')
    
    return model, scaler

def predict(features):
    # Cargamos el modelo y el scaler
    model = joblib.load('classifier_model.joblib')
    scaler = joblib.load('scaler.joblib')
    
    # Preparamos los datos
    features_array = np.array(features).reshape(1, -1)
    features_scaled = scaler.transform(features_array)
    
    # Realizamos la predicción
    prediction = model.predict(features_scaled)
    probability = model.predict_proba(features_scaled).max()
    
    return {
        'prediction': int(prediction[0]),
        'probability': float(probability)
    }

# Entrenamos el modelo
model, scaler = train_model()

## 3. Implementación de la API con FastAPI

Implementamos la API REST utilizando FastAPI:

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
from loguru import logger
import time

# Configuración del logger
logger.add("api.log", rotation="500 MB", level="INFO")

app = FastAPI(
    title="API de Predicción de Calidad del Vino",
    description="API REST para predecir la calidad del vino basado en sus características químicas",
    version="1.0.0"
)

class PredictionRequest(BaseModel):
    features: List[float] = None

    class Config:
        schema_extra = {
            "example": {
                "features": [
                    13.2,  # alcohol (% vol)
                    1.78,  # malic_acid (g/L)
                    2.14,  # ash (g/L)
                    11.2,  # alcalinity_of_ash (g/L)
                    100,   # magnesium (mg/L)
                    2.4,   # total_phenols (g/L)
                    2.4,   # flavanoids (g/L)
                    0.23,  # nonflavanoid_phenols (g/L)
                    1.71,  # proanthocyanins (g/L)
                    3.9,   # color_intensity (absorbancia)
                    1.3,   # hue (absorbancia)
                    2.5,   # OD280/OD315 of diluted wines
                    1100   # proline (mg/L)
                ]
            }
        }

class PredictionResponse(BaseModel):
    prediction: int
    probability: float
    prediction_time: float

@app.get("/")
def read_root():
    return {
        "message": "Bienvenido a la API de Predicción de Calidad del Vino",
        "endpoints": {
            "GET /": "Muestra esta información",
            "POST /predict": "Realiza una predicción de la calidad del vino",
            "GET /health": "Verifica el estado de la API"
        }
    }

@app.post("/predict", response_model=PredictionResponse)
def make_prediction(request: PredictionRequest):
    try:
        start_time = time.time()
        
        # Validamos que tengamos el número correcto de características
        if len(request.features) != 13:
            raise HTTPException(
                status_code=400,
                detail="Se requieren exactamente 13 características químicas del vino para la predicción"
            )
        
        # Realizamos la predicción
        result = predict(request.features)
        
        # Calculamos el tiempo de predicción
        prediction_time = time.time() - start_time
        
        # Registramos la predicción exitosa
        logger.info(
            f"Predicción exitosa - Clase: {result['prediction']}, "
            f"Probabilidad: {result['probability']:.4f}, "
            f"Tiempo: {prediction_time:.4f}s"
        )
        
        return PredictionResponse(
            prediction=result["prediction"],
            probability=result["probability"],
            prediction_time=prediction_time
        )
        
    except Exception as e:
        logger.error(f"Error en la predicción: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
def health_check():
    return {"status": "healthy", "timestamp": time.time()}

## 4. Iniciar el Servidor

Para iniciar el servidor FastAPI, ejecutamos:

In [ ]:
!uvicorn main:app --reload